# Comparison of statistics
Note : This notebook is not meant to be executed in the dima machine.

In this notebook, I am going to try normalization techniques on univariate statistics over numeric and categorical data in order to compare them. Then we will compare the different results of feature selection using these normalization techniques.

The statistics used here will be :
- Pearson's correlation
- Anova f-test

The comparison will be done with :
- No feature selection
- Min-max normalization and k-best selection
- k-best selection in numeric and categorical data independently

## Definition of the theoretical context
We consider a regression problem where $Y \sim \mathcal{N}(0,1)$ is the variable to predict.
The numeric variable are represented by the $N_i$. And the categorical variable are the $C_i$.
In order to be able to tune the correlation coefficient according to out needs, we will define the following distribution for numeric data : $N_i = Y + \varepsilon_i$ where $\varepsilon_i \sim \mathcal{N}(0, \sigma_i^2)$. Given the fact that $Y$ and $\varepsilon$ are independent, we reach with a few computation that : $r_{X,Y} = \dfrac{1}{\sqrt{1+\sigma_i^2}}$ where $ r_{X,Y}$ is the correlation coefficient between $X$ and $Y$.

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from augmentation.feature_selector import FeatureSelector

In [2]:
fs = FeatureSelector(numeric_stat="pearson", categoric_stat="anova")

In [3]:
# number of samples : n
n = 10000

Y = np.random.randn(n,1)
# px.histogram(x=Y)

In [4]:
def create_all_num_var(nb_num, max_sigma, target):
    # sigma = max_sigma*(np.random.rand(nb_num)) # for randomized sigma
    sigma = max_sigma*(np.linspace(0,1, nb_num)) # for deterministic sigma
    epsilon = np.random.randn(n, nb_num)
    N = target + epsilon*sigma
    return N

In [5]:
# number of numeric variables : nb_num
# range of the sigma_is is [0, max_sigma]
nb_num = 1000
max_sigma = 4

N = create_all_num_var(nb_num, max_sigma, Y)

Now that we have generated our numeric data, and verify that it is working as expected, let's generate the categoric data. As the computation of the anova f-value are not easy to manipulate, I will generate data with an underlying principle that takes into account what the anova accounts for. In other words, I will generate categorical that separates the target variable in group that behave more or less similarly. If we make them behave similarly, the anova value will decrease to 0, otherwise the anova value will increase.

The process for generating data is the following :

1. Gets the sorted index of the target variable $Y$ by target value
2. Splits this index in `num_var` sets of indices. `num_var` is a tunable parameters that sets the number of different values for one categorical variable.
3. Assign to each set of indices, a given value
4. For a proportion called `shuffle_proportion` assign a random value of the categorical variable

In [6]:
def create_cat_var(shuffle_proportion, num_var, target):
    n = target.shape[0]
    idx = np.argsort(target, axis=0)
    cat_var = np.zeros(n)
    split_idx = np.array_split(idx, num_var)
    for i, indice_array in enumerate(split_idx):
        indice_array.reshape((-1))
        for indice in indice_array:
            if np.random.rand() < shuffle_proportion:
                cat_var[indice] = np.random.randint(num_var)
            else:
                cat_var[indice] = i
    cat_var = cat_var.reshape((-1,1))
    return cat_var

In [7]:
def create_all_cat_var(nb_cat, min_cat, max_cat, target):
    # shuffle_proportion_array = np.random.rand(nb_cat) # for random shuffle proportion
    shuffle_proportion_array = np.linspace(0,1, nb_cat) # for random shuffle proportion
    num_cat_array = np.random.randint(min_cat, max_cat, size=(nb_cat))
    C = np.hstack((create_cat_var(shuffle_proportion, num_var, target) for shuffle_proportion, num_var in zip(shuffle_proportion_array, num_cat_array)))
    return C

In [8]:
# number of categorical variables : nb_cat
nb_cat = 100

# min and max number of categorical values for a categorical column : min_cat, max_cat
min_cat = 3
max_cat = 10

C = create_all_cat_var(nb_cat, min_cat, max_cat, Y)

Next plot verifies that the creation of the categorical data is corresponding to the idea described above.

In [9]:
# Testing that the creation of categorical variables is going as planned
cat1 = create_cat_var(0.5, 5, Y)
df = pd.DataFrame()
df["target"] = Y.reshape(-1)
df["cat1"] = cat1.reshape(-1)
px.histogram(df, "target", color="cat1")

After having generated the data with `create_all_num_var` and `create_all_cat_var`. I will define a function that will compute the statistics and plot their distributions.

In [10]:
def compute_stat(N, C, Y, feature_selector, show=False):
    corr = np.array([feature_selector.stat_numeric_numeric(pd.Series(Ni), Y.reshape((-1))) for Ni in N.T])
    anova = np.array([feature_selector.stat_numeric_categoric(pd.Series(Ci), pd.Series(Y.reshape((-1)))) for Ci in C.T])
    if show:
        fig = make_subplots(rows=2, cols=1)
        fig.add_trace(go.Histogram(x=corr, name="Distribution of the correlation"), row=1, col=1)
        fig.add_trace(go.Histogram(x=anova, name="Distribution of anova"), row=2, col=1)
        fig.show()
    return corr, anova

In [11]:
corr, anova = compute_stat(N, C, Y, fs)

As we want to test different strategies with different distribution, we may want to retrieve only a part of the variables we created, this can be done using the next function: `sub_sample`.

In [12]:
def sub_sample(N, C, corr, anova, min_corr, max_corr, min_anova, max_anova, show=False):
    N_idx = np.argwhere(np.logical_and(min_corr < corr, corr < max_corr)).reshape(-1)
    C_idx = np.argwhere(np.logical_and(min_anova < anova, anova < max_anova)).reshape(-1)
    new_corr, new_anova = corr[N_idx], anova[C_idx]
    if show:
        fig= make_subplots(rows=2, cols=1)
        fig.add_trace(go.Histogram(x=corr, name="Distribution of the old correlation", bingroup=1), row=1, col=1)
        fig.add_trace(go.Histogram(x=new_corr, name="Distribution of the new correlation", bingroup=1), row=1, col=1)
        fig.add_trace(go.Histogram(x=anova, name="Distribution of old anova", bingroup=2), row=2, col=1)
        fig.add_trace(go.Histogram(x=new_anova, name="Distribution of new anova", bingroup=2), row=2, col=1)
        fig.update_layout(barmode="overlay", bargap=0.1)
        fig.show()
    return N[:,N_idx], C[:,C_idx], new_corr, new_anova

In [13]:
_ = sub_sample(N,C, corr, anova, 0.4, 0.7, 1000, 4000, True)

## Feature selection
Above, we have defined our data : target variable $Y$, numeric and categorical variable : $N_i, C_i$.
Now, we will select the features with different strategies:

- k best features from each
- k best features with normalized statistics

In the next cell, I define the whole dataset, it is used for baseline purposes. On this whole dataset, we can perform linear regression directly or test wrapper methods to see how they perform compared to our normalization strategies.

In [14]:
# No feature selection
def df_no_selection(N,C,Y):
    df_N = pd.DataFrame(N)
    df_N.columns = [f"N_{col}" for col in df_N.columns]
    df_C = pd.DataFrame(C)
    df_C.columns = [f"C_{col}" for col in df_C.columns]
    df = pd.concat([df_N,df_C], axis=1)
    df["target"] = Y
    return df

In [15]:
df = df_no_selection(N,C,Y)
df.head()

,N_0,N_1,N_2,N_3,N_4,N_5,N_6,N_7,N_8,N_9,...,C_91,C_92,C_93,C_94,C_95,C_96,C_97,C_98,C_99,target
0,-0.219732,-0.214377,-0.208719,-0.238789,-0.237771,-0.233816,-0.214563,-0.214410,-0.185813,-0.267003,...,0.0,4.0,4.0,5.0,2.0,0.0,8.0,1.0,5.0,-0.219732
1,1.672267,1.670160,1.666040,1.674027,1.654947,1.664418,1.687467,1.658018,1.659191,1.662672,...,0.0,2.0,4.0,2.0,3.0,2.0,7.0,0.0,4.0,1.672267
2,0.519537,0.524068,0.523816,0.523633,0.531534,0.553224,0.506909,0.516863,0.548462,0.474203,...,3.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,5.0,0.519537
3,-0.192543,-0.193787,-0.205279,-0.194941,-0.207837,-0.223485,-0.214722,-0.232191,-0.179521,-0.202130,...,0.0,3.0,1.0,2.0,0.0,0.0,8.0,0.0,2.0,-0.192543
4,-0.178114,-0.180630,-0.178962,-0.178771,-0.181756,-0.192582,-0.208343,-0.205194,-0.171237,-0.149881,...,1.0,0.0,0.0,3.0,1.0,0.0,5.0,2.0,1.0,-0.178114


In [16]:
from augmentation.strategy import k_best_independent, k_best_normalized

In [17]:
def stat_and_type_dict(fs_corr, fs_anova):
    stat_dict = {
        0: {i: corr for i, corr in enumerate(fs_corr)},
        1: {i: anova for i, anova in enumerate(fs_anova)},
    }
    type_dict = {
        0: {i: "numeric" for i in range(nb_num)},
        1: {i: "categoric" for i in range(nb_cat)},
    }
    return stat_dict, type_dict

In [18]:
stat_dict, type_dict = stat_and_type_dict(corr, anova)

In [38]:
def select_independent(N,C,Y, stat_dict, type_dict, k_best=10, show=False):
    col_independent = k_best_independent(stat_dict, type_dict, 10)
    if show:
        print("Independently chosen columns are: ", col_independent)
    dict_independent = {
        "target": Y.reshape((-1)),
        **{f"N_{j}": N[:,j] for j in col_independent[0]},
        **{f"C_{j}": C[:,j] for j in col_independent[1]},
    }
    df_independent = pd.DataFrame(dict_independent)
    col_categoric = list(filter(lambda col: col[0] == "C", df_independent.columns))
    dummies = pd.get_dummies(df_independent[col_categoric].astype(int).astype(str))
    df_independent = df_independent.drop(columns=col_categoric)
    df_independent = pd.concat([df_independent, dummies], axis=1)
    return df_independent

In [20]:
df_independent = select_independent(N,C,Y, stat_dict, type_dict, show=True)

Independently chosen columns are:  {0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 1: [0, 1, 2, 4, 3, 5, 8, 6, 7, 11]}


In [39]:
def select_min_max_norm(N,C,Y, stat_dict, type_dict, k_best=20, show=False):
    col_normalized = k_best_normalized(stat_dict, type_dict, 20)
    if show:
        print("Normalized chosen columns are: ", col_normalized)
    dict_normalized = {
        "target": Y.reshape((-1)),
        **{f"N_{j}": N[:,j] for j in col_normalized[0]},
        **{f"C_{j}": C[:,j] for j in col_normalized[1]},
    }
    df_normalized = pd.DataFrame(dict_normalized)
    col_categoric = list(filter(lambda col: col[0] == "C", df_normalized.columns))
    dummies = pd.get_dummies(df_normalized[col_categoric].astype(int).astype(str))
    df_normalized = df_normalized.drop(columns=col_categoric)
    df_normalized = pd.concat([df_normalized, dummies], axis=1)
    return df_normalized

In [22]:
df_normalized = select_min_max_norm(N,C,Y, stat_dict, type_dict, show=True)

Normalized chosen columns are:  {0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18], 1: [0]}


## Training model
For the training of the model, I will use a simple Linear Regression model as it is well suited to the data we generated.

In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [24]:
def train_linear(df):
    prediction_col = df.columns.drop("target")
    X_train, X_test, y_train, y_test = train_test_split(df[prediction_col], df["target"])
    lin = LinearRegression().fit(X_train, y_train)
    mse = mean_squared_error(y_test, lin.predict(X_test), squared=False)
    return mse

In [25]:
mse = train_linear(df_independent)
print("MSE for independent strategy is : ", mse)
mse = train_linear(df_normalized)
print("MSE for normalized strategy is : ", mse)
mse = train_linear(df)
print("MSE for no selection strategy is : ", mse)

MSE for independent strategy is :  3.941886319481679e-16
MSE for normalized strategy is :  8.868988747942118e-16
MSE for no selection strategy is :  8.673541182037494e-16


In [26]:
def train_wrapper(df, sfs=None):
    prediction_col = df.columns.drop("target")
    X_train, X_test, y_train, y_test = train_test_split(df[prediction_col], df["target"])
    if not(sfs):
        sfs = SFS(
            LinearRegression(),
            k_features=10,
            forward=True,
            scoring="neg_mean_squared_error",
            verbose=1,
        )
        sfs.fit(X_train, y_train)
        print("Wrapper method chose the following features : ", sfs.k_feature_idx_)
    df_wrapped = pd.DataFrame(sfs.transform(df))
    df_wrapped["target"] = df["target"]
    mse = train_linear(df_wrapped)
    return mse, sfs

In [27]:
mse, sfs = train_wrapper(df)
print("MSE for wrapper strategy is : ", mse)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1100 out of 1100 | elapsed:   10.3s finished
Features: 1/10[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1099 out of 1099 | elapsed:   10.9s finished
Features: 2/10[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1098 out of 1098 | elapsed:   11.4s finished
Features: 3/10[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1097 out of 1097 | elapsed:   11.5s finished
Features: 4/10[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1096 out of 1096 | elapsed:   12.5s finished
Features: 5/10[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1095 out of 1095 | elapsed:   13.9s finished
Features: 6/

## Influence of the distribution
In the beginning of this notebook, we :

1. Created the dataset
1. Added the possibility to subsample the dataset to restrict the statistics values
1. Created the function that perform feature selection
1. Created function that allow us to train the model

Now we can go ahead and try to see what is the influence of the distribution of the statistics on the feature selection strategy.

In [53]:
min_corr_list = [0.,0.2,0.45]
max_corr_list = [0.55,0.7,0.9]
min_anova_list = [0.,200,500]
max_anova_list = [1000, 5000,10000]

In [54]:
from tqdm import tqdm_notebook as tqdm
from itertools import product

In [55]:
influence_list = []
t = tqdm(product(min_corr_list, max_corr_list, min_anova_list, max_anova_list), total = len(min_corr_list)*len(max_corr_list)*len(min_anova_list)*len(max_anova_list))
for min_corr, max_corr, min_anova, max_anova in t:
    # t.set_description(f"Correlation range: [{min_corr}, {max_corr}], Anova range: [{min_anova}, {max_anova}]")
    # t.refresh()
    sample_N, sample_C, sample_corr, sample_anova = sub_sample(N, C, corr, anova, min_corr, max_corr, min_anova, max_anova)
    # Testing with no selection
    df = df_no_selection(sample_N, sample_C, Y)
    mse = train_linear(df)
    influence_list.append([min_corr, max_corr, min_anova, max_anova, "no selection", mse])
    # Testing feature selection
    sample_stat_dict, sample_type_dict = stat_and_type_dict(sample_corr, sample_anova)
    df_independent = select_independent(sample_N, sample_C, Y, sample_stat_dict, sample_type_dict)
    mse = train_linear(df_independent)
    influence_list.append([min_corr, max_corr, min_anova, max_anova, "independent", mse])
    df_normalized = select_min_max_norm(sample_N, sample_C, Y, sample_stat_dict, sample_type_dict)
    mse = train_linear(df_normalized)
    influence_list.append([min_corr, max_corr, min_anova, max_anova, "normalized", mse])

In [66]:
influence_df = pd.DataFrame(influence_list, columns=["min_corr", "max_corr", "min_anova", "max_anova", "strategy", "mse"])
influence_df.head()

,min_corr,max_corr,min_anova,max_anova,strategy,mse
0,0.0,0.55,0.0,1000,no selection,0.099915
1,0.0,0.55,0.0,1000,independent,0.341290
2,0.0,0.55,0.0,1000,normalized,0.337161
3,0.0,0.55,0.0,5000,no selection,0.098158
4,0.0,0.55,0.0,5000,independent,0.241934


In [67]:
px.scatter(influence_df, x="mse", y="min_corr", color="strategy")

In [61]:
px.scatter(influence_df, x="mse", y="max_corr", color="strategy")

In [60]:
px.scatter(influence_df, x="mse", y="min_anova", color="strategy")

In [59]:
px.scatter(influence_df, x="mse", y="max_anova", color="strategy")

## Interpretation
As we are varying 5 parameters simultaneously (correlation range, anova range and feature selection strategy), it is not easy to visualize the results.
To be able to visualize the influence of each parameter, a solution can be to train a linear regression model on the influence of the different parameters so we could see which parameter is more influent.

In [68]:
dummy = pd.get_dummies(influence_df["strategy"])
influence_and_dummy = pd.concat([influence_df.drop(columns=["strategy"]), dummy], axis=1)
influence_and_dummy.head()

,min_corr,max_corr,min_anova,max_anova,mse,independent,no selection,normalized
0,0.0,0.55,0.0,1000,0.099915,0,1,0
1,0.0,0.55,0.0,1000,0.341290,1,0,0
2,0.0,0.55,0.0,1000,0.337161,0,0,1
3,0.0,0.55,0.0,5000,0.098158,0,1,0
4,0.0,0.55,0.0,5000,0.241934,1,0,0


In [85]:
# influence_df["min_anova"] = influence_df["min_anova"]/500
# influence_df["max_anova"] = influence_df["max_anova"]/10000
col_param = influence_df.columns.drop(["mse","strategy"])
influence_nof = LinearRegression(normalize=True).fit(influence_df[influence_df["strategy"] == "no selection"][col_param], influence_df[influence_df["strategy"] == "no selection"]["mse"])
influence_indep = LinearRegression(normalize=True).fit(influence_df[influence_df["strategy"] == "independent"][col_param], influence_df[influence_df["strategy"] == "independent"]["mse"])
influence_norm = LinearRegression(normalize=True).fit(influence_df[influence_df["strategy"] == "normalized"][col_param], influence_df[influence_df["strategy"] == "normalized"]["mse"])
influence_nof.coef_

array([ 0.04862081, -0.18257163,  0.00070807, -0.00260581])

In [100]:
px.bar(y=col_param ,x=influence_nof.coef_, orientation="h", title="Influence of range with no feature selection", labels={"x":"mse", "y":""}, range_x=[-0.6,0.1])

In [102]:
px.bar(y=col_param ,x=influence_indep.coef_, orientation="h", title="Influence of range with independent feature selection", labels={"x":"mse", "y":""}, range_x=[-0.6,0.1])

In [101]:
px.bar(y=col_param ,x=influence_norm.coef_, orientation="h", title="Influence of range with normalized feature selection", labels={"x":"mse", "y":""}, range_x=[-0.6,0.1])